In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import torch 
from argparse import Namespace
from tqdm import tqdm
import pickle 
import glob 
import ast

# Add the path to the directory containing the sybil module
sys.path.append('/workspace/home/tengyuezhang/sybil_cect/code/Sybil/')
from sybil.utils.metrics import concordance_index, get_survival_metrics
from sybil import Sybil, Serie
from sybil import visualize_attentions_v2


In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
num_threads = os.cpu_count() // 5

In [9]:
ucla_ldct_cases_path = '/workspace/home/tengyuezhang/sybil_cect/data/ucla_ldct/ucla_ldct_20_cases_baseline_tp.csv'
output_path = '/workspace/home/tengyuezhang/sybil_cect/results/ucla_ldct/ucla_ldct_20_risk_scores.csv'
vis_dir_path = "/workspace/home/tengyuezhang/sybil_cect/visualizations/ucla_ldct_20_attention_maps"
save_atten_maps = True
if not os.path.exists(vis_dir_path):
    os.makedirs(vis_dir_path)

In [10]:
# Initialize the Sybil model
model = Sybil("sybil_ensemble")
num_years = 6

/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tengyuezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Load the CSV file
all_cases = pd.read_csv(ucla_ldct_cases_path)
df = all_cases

In [15]:
for i in range(num_years):
    df[f'pred_risk_year_{i}'] = np.nan

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing cases"):
   
    dicom_dir = row['Directory']
    # event = row['LungCancer']
    event = 0
    years_to_event = 1
    pid = row['pid']
    dicom_list = glob.glob(dicom_dir + '/*')
    serie = Serie(dicom_list, label=event, censor_time=years_to_event)
    
    results = model.predict([serie], return_attentions=True, threads=num_threads)
        
    # Update the risk scores columns for the current row
    for i in range(num_years):
        df.at[index, f'pred_risk_year_{i}'] = results.scores[0][i]
        
    # Save the updated DataFrame to the output CSV file at each iteration
    df.to_csv(output_path, index=False)
    
    # Save attention maps 
    if save_atten_maps: 
        attentions = results.attentions

        series_with_attention = visualize_attentions_v2(
            serie,
            attentions = attentions,
            pid = pid, 
            save_directory = os.path.join(vis_dir_path, str(pid)),
            gain = 1, 
            save_pngs = True, 
            save_rep_slice = True,
            save_bbox = False, 
            centers = None, 
            nodule_slices = [], 
            nodule_labels = [],
        )

Processing cases: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [1:12:14<00:00, 216.71s/it]
